In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/home/fahd/Documents/headline-trader/headline-trader/src/training/full_df.csv')
df.head()

,timestamp,title,price,sentiment,sentiment_score,future_price,label,symbol,hour_of_day,day_of_week
0,2025-04-25 14:51:09.451590+00:00,Cryptocurrency campaigners call for Swiss cent...,95478.36,neutral,0.854923,94933.07,sell,BTCUSDT,14,4
1,2025-04-25 14:51:10.891138+00:00,The Fed Just Quietly Primed Bitcoin And Crypto...,95478.36,neutral,0.902940,94933.07,sell,BTCUSDT,14,4
2,2025-04-25 14:51:12.271174+00:00,Bitcoin ETFs Gain as Dollar Trust Falters - ET...,95478.36,negative,0.740927,94933.07,sell,BTCUSDT,14,4
3,2025-04-25 14:51:13.556977+00:00,Bitcoin – a certain kind of hedge? - DWS Asset...,95478.36,neutral,0.939790,94933.07,sell,BTCUSDT,14,4
4,2025-04-25 14:51:14.739492+00:00,Bitcoin gets complicated as Tether and SoftBan...,95478.36,neutral,0.910740,94933.07,sell,BTCUSDT,14,4


In [3]:
label_map = {'buy': 0, 'hold': 1, 'sell': 2}
df['label'] = df['label'].map(label_map)


# Features & labels
features = df[['price', 'sentiment_score', 'hour_of_day', 'day_of_week']].values
labels = df['label'].values

features, labels

(array([[9.54783600e+04, 8.54923427e-01, 1.40000000e+01, 4.00000000e+00],
        [9.54783600e+04, 9.02940094e-01, 1.40000000e+01, 4.00000000e+00],
        [9.54783600e+04, 7.40927398e-01, 1.40000000e+01, 4.00000000e+00],
        ...,
        [2.19420000e+00, 9.08314943e-01, 2.00000000e+00, 5.00000000e+00],
        [2.19420000e+00, 8.60967100e-01, 2.00000000e+00, 5.00000000e+00],
        [2.19420000e+00, 9.38049912e-01, 2.00000000e+00, 5.00000000e+00]],
       shape=(6723, 4)),
 array([2, 2, 2, ..., 1, 1, 1], shape=(6723,)))

In [4]:
# Normalize
scaler = StandardScaler()
features = scaler.fit_transform(features)


# convert to pytorch tensors
X = torch.tensor(features, dtype=torch.float32)
y = torch.tensor(labels, dtype=torch.long)

X.shape[0]

6723

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [6]:
# Using DataLoader for batching 

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)


In [7]:
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(32, 3)  # 3 classes: buy/sell/hold

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

model = MyModel(input_size=X.shape[1])

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

In [9]:
epochs = 20
for epoch in range(epochs):
    
    # --- Training Phase ---
    model.train()  # important: sets model to training mode
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # --- Validation Phase ---
    model.eval()  # important: disables dropout, batchnorm, etc.
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # no gradient calculations
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    val_accuracy = correct / total
    print(f"Epoch {epoch}, Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {val_loss:.4f}")


Epoch 0, Validation Accuracy: 0.5621, Validation Loss: 35.1993
Epoch 1, Validation Accuracy: 0.5011, Validation Loss: 32.6317
Epoch 2, Validation Accuracy: 0.5643, Validation Loss: 32.4291
Epoch 3, Validation Accuracy: 0.6796, Validation Loss: 31.2085
Epoch 4, Validation Accuracy: 0.6773, Validation Loss: 29.7628
Epoch 5, Validation Accuracy: 0.6446, Validation Loss: 30.6737
Epoch 6, Validation Accuracy: 0.6959, Validation Loss: 27.0074
Epoch 7, Validation Accuracy: 0.6959, Validation Loss: 26.7558
Epoch 8, Validation Accuracy: 0.6959, Validation Loss: 26.4136
Epoch 9, Validation Accuracy: 0.6959, Validation Loss: 26.6276
Epoch 10, Validation Accuracy: 0.6944, Validation Loss: 26.5760
Epoch 11, Validation Accuracy: 0.6959, Validation Loss: 26.4612
Epoch 12, Validation Accuracy: 0.6959, Validation Loss: 26.6560
Epoch 13, Validation Accuracy: 0.6929, Validation Loss: 26.4727
Epoch 14, Validation Accuracy: 0.6959, Validation Loss: 26.3343
Epoch 15, Validation Accuracy: 0.6959, Validation 

In [10]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

print(f"Accuracy: {100 * correct / total:.2f}%")

Accuracy: 70.41%


In [ ]:
torch.save(model.state_dict(), "model.pth")